In [1]:
import joblib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [ ]:
with open(r'/Users/alexanderhsu/Google Drive/My Drive/2023Data/asoid_manuscript/active_learning_fixed_2newgroups.sav', 'rb') as fr:
    [X_train_list, Y_train_list,
     iterX_f1_scores_list, iterX_macro_scores_list, iterX_predict_prob_list,
     sampled_idx_list] = joblib.load(fr)
with open(r'/Users/alexanderhsu/Google Drive/My Drive/2023Data/asoid_manuscript/using_all_train_2newgroups.sav', 'rb') as fr:
    [subsampled_features_list, subsampled_targets_list,
     remaining_features_list, remaining_targets_list,
     peak_f1_scores, peak_macro_scores, peak_predict_prob] = joblib.load(fr)

In [ ]:
f1_ = []
macro_avg_ = []
for j in range(len(iterX_f1_scores_list)):
    f1_.append(np.array(iterX_f1_scores_list[j]))
    macro_avg_.append(np.array(iterX_macro_scores_list[j]))

In [ ]:
behavior_types = ['Average', 'Attack', 'Investigation', 'Mount', 'Anogential1', 'Anogential2']
cmap_beh = ['crimson', 'darkorange', 'steelblue', 'plum', 'goldenrod']
fig, ax = plt.subplots(1, 1, figsize=(4, 3))
# plot error bar with line for asoid indiv f1
for b in range(len(cmap_beh)):
    x = np.arange(len(f1_))
    y = np.median(f1_, axis=1)[:, b]
    y_err = np.std(macro_avg_, axis=1)
    ax.errorbar(x, y, yerr=y_err, barsabove=True, lw=3,
                color=cmap_beh[b], ecolor=cmap_beh[b], alpha=0.8, elinewidth=3)
# in terms of macro f1
x_all = np.arange(len(macro_avg_))
y_all = np.mean(macro_avg_, axis=1)
std_all = np.std(macro_avg_, axis=1)
plot, caps, bar = ax.errorbar(x_all, y_all, yerr=std_all, barsabove=True, lw=3,
                              color='k', ecolor='k', alpha=0.8, elinewidth=3)

ax.set_xticks(np.arange(4, len(macro_avg_), 5))
ax.set_xticklabels(np.arange(5, len(macro_avg_)+1, 5))
ax.set_ylabel('F1 score')
ax.set_xlabel('Active learning iteration #')
ax.set_ylim([0.65, 0.95])
# plot performance line using all data
ax.hlines(np.median(peak_macro_scores, axis=0),
          0,
          len(macro_avg_)-1,
          ls='--',
          color='k')
for b in range(len(cmap_beh)):
    ax.hlines(np.median(peak_f1_scores, axis=0)[b],
              0,
              len(macro_avg_)-1,
              ls='--',
              color=cmap_beh[b])
ax.set_xlim([-.5, len(macro_avg_)-.5])
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.legend(behavior_types)
plt.show()

In [ ]:
behavior_types = ['Average', 'Attack', 'Investigation', 'Mount', 'Anogential1', 'Anogential2']
## plot average counts across 20 seeds, and compare with all avail training
fig, ax = plt.subplots(1, 1, figsize=(4, 1))
class_counts_per = [np.hstack([np.mean([len(np.where(Y_train_list[it][i]==b)[0])
                                        for i in range(len(Y_train_list[it]))])
                               for it in range(len(Y_train_list))])
                    for b in np.unique(Y_train_list[0][0])]

class_counts_total = [len(np.where(subsampled_targets_list[0][subsampled_targets_list[0] < 5] == b)[0])
                      for b in np.unique(subsampled_targets_list[0]) if b < 5]

df_all = pd.DataFrame(np.vstack((np.array(class_counts_per).T, np.array(class_counts_total))))
df_all.plot(kind='bar', stacked=True, legend=False, color=cmap_beh, ax=ax)
ax.set_xticklabels('')
ax.set_yticklabels('')
ax.set_xticks(np.hstack((np.arange(4, 20, 5), 20)))
ax.set_yticks(np.arange(2000, 15000, 6000))
ax.legend(behavior_types)
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
plt.show()

In [ ]:
behavior_types = ['Average', 'Attack', 'Investigation', 'Mount', 'Anogential1', 'Anogential2']
## plot average counts across 20 seeds
fig, ax = plt.subplots(1, 1, figsize=(4, 1))
df_activeonly = pd.DataFrame(np.vstack((np.array(class_counts_per).T)))
# stack bar
df_activeonly.plot(kind='bar', stacked=True, legend=False, color=cmap_beh, ax=ax)
ax.set_xticklabels('')
ax.set_yticklabels('')
ax.set_xticks(np.hstack((np.arange(4, 20, 5), 20)))
ax.set_yticks(np.arange(0, 3000, 1000))
ax.legend(behavior_types)
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
plt.show()